In [1]:
from model import Transformer
from config import Config
from tokenizer import Tokenizer
import torch as t
import os
import warnings

warnings.filterwarnings("ignore")

Using device: cuda
fixed_digit: False
Training for 3000 epochs
Batch size: 256
Learning rate: 0.001
Train on 0.5 of the data
Saving model every 5 epochs


In [2]:
def get_model_folder_at_index(model_dir, file_start, folder_index):
    model_files = [file for file in os.listdir(model_dir) if file_start in file]
    model_files.sort()
    return os.path.join(model_dir, model_files[folder_index])


In [3]:
model_dir = 'saved_runs'
file_start = 'mod_digit_add_'
config = Config()
folder_index = -1
latest_model_folder = get_model_folder_at_index(model_dir, file_start, folder_index) 
latest_model_folder 

'saved_runs/mod_digit_add_2024-10-03_17-00-11'

In [4]:
# total models will be num_models + 1
num_models = 10
num_epochs = 3000

first_model = Transformer(config)
first_model.load_state_dict(t.load(latest_model_folder + '/init.pth')['model'])
first_model.eval()


models = [Transformer(config) for _ in range(num_models-1)]
intervals = t.linspace(0, num_epochs, num_models + 1).int()[1:-1]
print(intervals)
weights = [t.load(latest_model_folder + f'/models/{num}.pth') for num in intervals] 
for i in range(len(weights)):
    models[i].load_state_dict(weights[i]['model'])
    models[i].eval()

last_model = Transformer(config)
last_model.load_state_dict(t.load(latest_model_folder + f'/final.pth')['model'])
last_model.eval()

tokenizer = Tokenizer(config)

tensor([ 300,  600,  900, 1200, 1500, 1800, 2100, 2400, 2700],
       dtype=torch.int32)


In [5]:
def get_accuracy(model, maxnum = 100):

    correct = 0
    count = 0
    for i in range(maxnum):
        for j in range(maxnum):
            count += 1
            print(f"Accuracy: {(correct/count):.2%}, Count: {count}", end = '\r')
            correct_answer = (i + j)
            lsi = [int(k) for k in str(i)]
            lsj = [int(k) for k in str(j)]
            question = lsi + [10] + lsj + [11]

            ll = len(question)
            #print("q:", question)
            pred = model.generate_greedy(question)
            answer = pred[ll: -1]
            try:
                answer = int(tokenizer.detokenize(answer))
            except:
                continue
            #print("p", pred)
            #print("a", answer)

            if answer == correct_answer:
                correct += 1
            
            
    print('\n')

In [6]:
print("Calculating accuracy for the initial model")
get_accuracy(first_model)

for i, model in enumerate(models):
    print(f"Calculating accuracy for model {intervals[i]}")
    get_accuracy(model)

print("Calculating accuracy for the final model")
get_accuracy(last_model)

Calculating accuracy for the initial model
Accuracy: 0.00%, Count: 10000

Calculating accuracy for model 300
Accuracy: 98.89%, Count: 10000

Calculating accuracy for model 600
Accuracy: 98.60%, Count: 10000

Calculating accuracy for model 900
Accuracy: 99.17%, Count: 10000

Calculating accuracy for model 1200
Accuracy: 99.14%, Count: 10000

Calculating accuracy for model 1500
Accuracy: 99.46%, Count: 10000

Calculating accuracy for model 1800
Accuracy: 99.45%, Count: 10000

Calculating accuracy for model 2100
Accuracy: 99.21%, Count: 10000

Calculating accuracy for model 2400
Accuracy: 99.62%, Count: 10000

Calculating accuracy for model 2700
Accuracy: 99.66%, Count: 10000

Calculating accuracy for the final model
Accuracy: 99.62%, Count: 10000



In [1]:
test_sentence = "1+81="
test_sentence_tokenized = tokenizer.tokenize(test_sentence)

pred = model.generate_greedy([1, 10, 8, 1, 11])
print("Prediction:", pred)

NameError: name 'tokenizer' is not defined

In [8]:
test_sentence_tokenized

[1, 10, 1, 11]